#  Data Analysis Tools JWebbinar: Specutils

![Specutils: An Astropy Package for Spectroscopy](specutils_logo.png)


This notebook provides an overview of the Astropy coordinated package `specutils`.  While this notebook is intended as an interactive introduction to specutils at the time of its writing, the canonical source of information for the package is the latest version's documentation: 

https://specutils.readthedocs.io

`specutils` should already be in your JWebbinar environment.  If you wish to install locally, you can follow the instructions in [the installation section of the specutils docs](https://specutils.readthedocs.io/en/latest/installation.html).

Once specutils is installed, fundamental imports necessary for this notebook are possible:

In [ ]:
import numpy as np

import astropy.units as u
from astropy.io import fits

import specutils
from specutils import Spectrum1D, SpectralRegion, analysis, manipulation, fitting
specutils.__version__

In [ ]:
# for plotting:
%matplotlib inline
import matplotlib.pyplot as plt


# for showing quantity units on axes automatically:
from astropy.visualization import quantity_support
quantity_support();

# Background/Spectroscopic ecosystem

The large-scale plan for spectroscopy support in the Astropy project is outlined in  the [Astropy Proposal For Enhancement 13](https://github.com/astropy/astropy-APEs/blob/main/APE13.rst).  In summary, this APE13 lays out three broad packages:

* `specutils` - a Python package containing the basic data structures for representing spectroscopic data sets, as well as a suite of fundamental spectroscopic analysis tools to work with these data structures.
* `specreduce` - a general Python package to reduce raw astronomical spectral images to 1d spectra (represented as `specutils` objects).
* `specviz` - a Python package (or possibly suite of packages) for visualization of astronomical spectra.


While all are still in development, the first of these is furthest along, and is the subject of this notebook, as it contains the core data structures and concepts required for the others.

# Fundamentals of specutils

## Objects for representing spectra

The most fundamental purpose of `specutils` is to contain the shared Python-level data structures for storing astronomical spectra.  It is important to recognize that this is not the same as the *on-disk* representation.  As desecribed later specutils provides loaders and writers for various on-disk representations, with the intent that they all load to a common set of in-memory/Python interfaces.  Those intefaces (implemented as Python classes) are described in detail in the [relevant section of the documentation](https://specutils.readthedocs.io/en/latest/types_of_spectra.html), which contains this diagram:

![Specutils Classes](specutils_classes_diagrams.png)

The core principal is that all of these representations contain a `spectral_axis` attribute as well as a `flux` attribute (as well as optional matching `uncertainty`).  The former is often wavelength for OIR spectra, but might be frequency or energy for e.g. Radio or X-ray spectra.  Regardless of which spectral axis is used, the class attempts to interpret it appropriately, using the features of `astropy.Quantity` to distinguish different types of axes.  Similarly, `flux` may or may not be a traditional astronomical `flux` unit (e.g. Jy or  erg sec$^{-1}$ cm$^{-2}$ angstrom$^{-1}$), but is treated as the portion of the spectrum that acts in that manner.  The various classes are then distinguished by whether these attributes are one-dimensional or not, and how to map the `spectral_axis` dimensionality onto the `flux`.  The simplest case (and the one primarily considered here) is the scalar `Spectrum1D` case, which is a single spectrum with a matched-size `flux` and `spectral_axis`.

## Basics of creating Spectrum1D Objects

If your spectrum is in a format that specutils understands, loading it is very straightforward.  There are times when you want a bit more control, though, so lets look at loading from a file and creating an object directly from arrays:

## Loading spectra from files

Specutils comes with readers for a variety of spectral data formats (including loaders for future JWST instruments). While support for specific formats depends primarily from users (like you!) providing readers, you may find that one has already been implemented for your favorite spectrum format.  As an example, we consider a simulated high-redshift (z > 1) galaxy like that you might see from NIRSpec:

In [ ]:
from astropy.utils.data import download_file

spec_fn = download_file('https://stsci.box.com/shared/static/b22b1fzhimtdqfp8597m4bg67kovvauu.fits', cache=True)

In [ ]:
jwst_spec = Spectrum1D.read(spec_fn)
plt.plot(jwst_spec.spectral_axis, jwst_spec.flux);

To see the full list of formats readable in your current version of specutils, see the table at the bottom of the `Spectrum1d.read` method:

In [ ]:
help(Spectrum1D.read)

### Exercise

If you have your own spectroscopic data, try loading a file here using either one of the built-in loaders, or the `Spectrum1D` interface, and plotting it.  If you don't have your own data on-hand, try downloading something of interest via a public archive (e.g., public HST data using MAST, or [an sdss galaxy](https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=1323&mjd=52797&fiberid=12)), and load it.

# Creating a Spectrum "by-hand"

If you have a format that is not compatible, or you want to do some sort of customization of the loading process, spectra can be created directly from aastropy quantities (which are basically arrays with associated units). Here we'll show how you can do that, using data from the same file above: 

In [ ]:
fitsfile = fits.open(spec_fn)
fitsfile.info()

In [ ]:
 fitsfile[1].header

In [ ]:
table_data = fitsfile[1].data
table_data

In [ ]:
flux = table_data['flux']*u.uJy
wavelength = table_data['wavelength']*u.um

spec1d_byhand = Spectrum1D(spectral_axis=wavelength, flux=flux)

plt.step(spec1d_byhand.spectral_axis, spec1d_byhand.flux)

spec1d_byhand

## Working with Units and Spectral Axes 

We created `Spectrum1D` just as Quantity arrays, so they can be treated just as `Quantity` objects when convenient with unit conversions and the like:

In [ ]:
jwst_spec.spectral_axis

In [ ]:
jwst_spec.spectral_axis.to(u.angstrom)

In [ ]:
jwst_spec.spectral_axis.to(u.THz, u.spectral())

There's even fast-accessors to make some of this more convenient:

In [ ]:
plt.step(jwst_spec.frequency, jwst_spec.flux)

But under the hood this are are fully-featured WCS following the [Astropy APE14](https://github.com/astropy/astropy-APEs/blob/main/APE14.rst) WCS interface along with the [GWCS](https://gwcs.readthedocs.io/) package. So you can use that to do conversions to and from spectral to pixel axes:

In [ ]:
jwst_spec.wcs.pixel_to_world([10, 10.5])

In [ ]:
jwst_spec.wcs.world_to_pixel([1.2, 1.3]*u.um)

The other dimension is the flux - that requires slightly more complex transformation because it matters where in the spectrum you are to do the flux transformation:

In [ ]:
f_lamb_units = u.erg / u.s / (u.cm**2) / u.um

jwst_spec.flux.to(f_lamb_units, u.spectral_density(jwst_spec.spectral_axis))

But specutils makes this much easier!

In [ ]:
jwst_spec_flamb = jwst_spec.new_flux_unit(f_lamb_units)
jwst_spec_flamb

In [ ]:
plt.plot(jwst_spec_flamb.spectral_axis, jwst_spec_flamb.flux)

## Uncertainties

Currently the most compatible way to use uncertainties is the machinery built for the `astropy.nddata` object (although in some cases simply passing in an uncertainty array will also work):

In [ ]:
from astropy.nddata import StdDevUncertainty

unc = StdDevUncertainty(0.05 * np.ones_like(jwst_spec.flux))
spec1d_unc = Spectrum1D(spectral_axis=jwst_spec.spectral_axis, 
                        flux=jwst_spec.flux, 
                        uncertainty=unc)

plt.fill_between(spec1d_unc.spectral_axis, 
                 spec1d_unc.flux - spec1d_unc.uncertainty.quantity, 
                 spec1d_unc.flux + spec1d_unc.uncertainty.quantity);

# Arithmetic on Spectra

Specutils provides a lot of functionality for manipulating spectra.  In general these follow the pattern of creating *new* specutils objects with the results of the operation instead of in-place operations.

The most straightforward of operations are arithmetic manipulations.  In general these follow patterns that are based on fundametal arithmetic. E.g.:

In [ ]:
newspec1d = jwst_spec - 0.5*u.uJy
plt.step(newspec1d.wavelength, newspec1d.flux)

However, when there is ambiguity in your intent - for example, two spectra with different units where it is not clear what the desired output is - errors are generally produced instead of the code attempting to guess:

In [ ]:
# This raises an error:
newspec1d - jwst_spec_flamb 

Resolving this requires explicit conversion:

In [ ]:
newspec1d.new_flux_unit(f_lamb_units) - jwst_spec_flamb

# Sample Analysis: Line Center for Redshift

Specutils has a large set of analysis functions, more than we have time to cover here.  So instead we focus on a specific concrete science case: determining the center of a line and using that to get a redshift estimate.

If you are used to looking at galaxies, you probably recognized that our example spectrum has a characteristic emission line pattern with a strong H$\alpha$ line. While we might be able to center up on a line that strong, to be sure we should always subtract the continuum first.  Many other `specutils` analaysis function expect continuum-subtracted spectra as well.

In [ ]:
# note this doesn't error because with a *scalar* it's unambiguous that the user wants the spectrum units
jwst_continuum = 550*u.nJy 

jwst_halpha_contsub = jwst_spec - jwst_continuum

plt.step(jwst_halpha_contsub.spectral_axis, jwst_halpha_contsub.flux)
plt.axhline(0, c='k', ls=':')

While there are techniques for identifying the line automatically (see the fitting section below), here we assume we are doing "quick-look" procedures where manual identification is possible. 

We do this by defining a `SpectralRegion` object spanning the area of the line.  Further below is a worked example of how to get these values easily in a notebook, but for now we can take the numbers I've pre-eyeballed: $1.638 \mu m$ to $1.644 \mu m$

In [ ]:
halpha_lines_region = SpectralRegion(16380*u.angstrom, 16440*u.angstrom)

plt.step(jwst_halpha_contsub.spectral_axis, jwst_halpha_contsub.flux)

yl1, yl2 = plt.ylim()
plt.fill_between([halpha_lines_region.lower, halpha_lines_region.upper], 
                 -1, 10, alpha=.2)
plt.xlim(1.6, 1.7);
plt.ylim(-.2, 2.5)

You can now call a variety of analysis functions on the continuum-subtracted spectrum to estimate various properties of the line (you can see the full list of relevant analysis functions [in the analysis part of the specutils docs](https://specutils.readthedocs.io/en/stable/analysis.html#functions)). Here we do just the center:

In [ ]:
cen = analysis.centroid(jwst_halpha_contsub, halpha_lines_region)
cen

Now we simply plug this into the redshift formula for H$\alpha$'s rest wavelength:

In [ ]:
(cen/(6563*u.angstrom)) - 1

We now have strong evidence this is a manufactured spectrum! (it's suspiciously close to *exactly* 1.5 ...)

# Additional Materials

From here on this notebook provides more in-depth details about the.  While there is likely not enough time to work through this during the Webbinar, it is provided for you to work through whichever parts might appeal to your workflows. This is the intended mode of working with `specutils`: it is a toolbox for you to build your own science cases!


## Sample Spectrum and SNR

Lets start with a simple calculation: the S/N of this spectrum. While pipeline-output JWST files will have uncertainties, for this example we are using a basic simulation without the uncertainities. Hence we start using an SNR estimate that follows a straightofrward algorithm detailed in the literature.

In [ ]:
analysis.snr_derived(jwst_spec)

Ideally we'd use an uncertainty *we* understand to derive the S/N:

In [ ]:
analysis.snr(jwst_spec)

But we need to define the uncertainty first!  To do that we need to specify the region to compute the uncertainty:

## Spectral Regions

Most analysis required on a spectrum requires specification of a part of the spectrum - e.g., a spectral line.  Because such regions may have value independent of a particular spectrum, they are represented as objects distrinct from a given spectrum object.  Below we outline a few ways such regions are specified.

In [ ]:
ha_region = SpectralRegion((6563-50)*u.AA, (6563+50)*u.AA)
ha_region

Regions can also be raw pixel values (although of course this is more applicable to a specific spectrum):

In [ ]:
ha_pixel_region = SpectralRegion(2650*u.pixel, 2850*u.pixel)
ha_pixel_region

Additionally, *multiple* regions can be in the same `SpectralRegion` object. This is useful for e.g. measuring multiple spectral features in one call:

In [ ]:
HI_wings_region = SpectralRegion([(1.44*u.GHz, 1.43*u.GHz), (1.41*u.GHz, 1.4*u.GHz)])
HI_wings_region

These regions are used for many other analysis steps, including, for example, specifying a "featureless" region to use to estimate a noise level to assume for the spectrum:

In [ ]:
noise_region = SpectralRegion(1.4*u.micron, 1.5*u.micron)
jwst_spec_with_unc = manipulation.noise_region_uncertainty(jwst_spec, noise_region)
jwst_spec_with_unc

With that uncertainty present, we can now compute the S/N:

In [ ]:
analysis.snr(jwst_spec_with_unc)

Regions can also be used to  they can be used to extract sub-spectra from larger spectra, which can be used to do yet further operations (like in this case, per-pixel S/N):

In [ ]:
jwst_spec_with_unc_ha = manipulation.extract_region(jwst_spec_with_unc, ha_pixel_region)
plt.step(jwst_spec_with_unc_ha.spectral_axis, 
         jwst_spec_with_unc_ha.flux/jwst_spec_with_unc_ha.uncertainty.quantity)

analysis.snr(jwst_spec_with_unc_ha)

## Line Measurements

While line-fitting (detailed more below) is a good choice for high signal-to-noise spectra or when detailed kinematics are desired, more empirical measures are often used in the literature for noisier spectra or just simpler analysis procedures. Specutils provides a set of functions to provide these sorts of measurements, as well as similar summary statistics about spectral regions.  The [analysis part of the specutils documentation](https://specutils.readthedocs.io/en/latest/analysis.html) provides a full list and detailed examples of these, but here we demonstrate some example cases.

Note: these line measurements generally assume your spectrum is continuum-subtracted or continuum-normalized. Some spectral pipelines do this for you, but often this is not the case.  For our examples here we will do this step "by-eye", but for a more detailed discussion of continuum modeling, see the next section.  Based on the plots above we estimate a continuum level for the area of the simulated JWST spectrum around the H-alpha emission line, and use basic math to construct the continuum-normalized and continuum-subtracted spectra.

In [ ]:
# estimate a reasonable continuum-level estimate for the h-alpha area of the spectrum
jwst_continuum = 550*u.nJy # note this is more convenient units than the spectrum itself

jwst_halpha_contsub = manipulation.extract_region(jwst_spec, ha_pixel_region) - jwst_continuum

plt.axhline(0, c='k', ls=':')
plt.step(jwst_halpha_contsub.spectral_axis, jwst_halpha_contsub.flux)

With the continuum level identified, we can now make some measurements of the spectral lines that are apparent by eye - in particular we will focus on the H-alpha emission line. While there are techniques for identifying the line automatically (see the fitting section below), here we assume we are doing "quick-look" procedures where manual identification is possible. 

In the cell below, change the values for `LOWER` and `UPPER` to make a spectral region that just encompasses the  H-alpha line (the middle of the three lines). You may find it useful to change the values, re-run the cell, and change again to "hone in" on the right number.

In [ ]:
LOWER = 16000 * u.angstrom
UPPER = 17000 * u.angstrom
halpha_lines_region = SpectralRegion(LOWER, UPPER)

plt.step(jwst_halpha_contsub.spectral_axis, jwst_halpha_contsub.flux)

yl1, yl2 = plt.ylim()
plt.fill_between([halpha_lines_region.lower, halpha_lines_region.upper], 
                 yl1, yl2, alpha=.2)
plt.ylim(yl1, yl2)
plt.xlim(1.6, 1.7);

You can now call a variety of analysis functions on the continuum-subtracted spectrum to estimate various properties of the line (you can see the full list of relevant analysis functions [in the analysis part of the specutils docs](https://specutils.readthedocs.io/en/stable/analysis.html#functions)). Here we highlight the line center, flux, and equivalent width.

In [ ]:
cen = analysis.centroid(jwst_halpha_contsub, halpha_lines_region)
cen

The line flux is equally straightforward, although deceptively complicated:

In [ ]:
analysis.line_flux(jwst_halpha_contsub, halpha_lines_region)

While technically correct, those are not necessarily the kinds of units you would need to compare with, say, optical line fluxes.  Fortunately, `Spectrum1D` objects provide straightforward unit conversion:

In [ ]:
f_lamb_unit = u.erg*u.m**-2*u.s**-1*u.micron**-1
jwst_halpha_contsub_flamb = jwst_halpha_contsub.new_flux_unit(f_lamb_unit)

analysis.line_flux(jwst_halpha_contsub_flamb, halpha_lines_region).to(u.erg/u.s/u.cm**2)

Equivalent width, being a continuum dependent property, can be computed directly from the spectrum if the continuum level is given:

In [ ]:
analysis.equivalent_width(jwst_spec, jwst_continuum, regions=halpha_lines_region).to(u.angstrom)

As per the normal convention, it is negative for an emission line.

## Exercises

Suppose you wanted to compare some of the above line measurements to a relatively local galaxy that has known line Hα line fluxes - say, some galaxy you know to be at 100 Mpc. How can you do this comparison in just a few lines of code? (Hint: making use of the units in the ``line_flux(...)`` outputs and [astropy.cosmology](https://docs.astropy.org/en/stable/cosmology/index.html) are your friends here!)  If you'd like to try this with some real data, you might compare to a ground-based spectrum of a local emission line galaxy, [like the SDSS](https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=1323&mjd=52797&fiberid=12).

Load one of the spectrum datasets you made in the overview exercises into this notebook (i.e., your own dataset, a downloaded one, or the blackbody with an artificially added spectral feature).  Make a flux or width measurement of a line in that spectrum directly (i.e. without subtracting a continuum).  Is anything odd?

## Continuum Subtraction

While continuum-fitting for spectra is sometimes thought of as an "art" as much as a science, specutils provides the tools to do a variety of approaches to continuum-fitting, without making a specific recommendation about what is "best" (since it is often very data-dependent).  More details are available [in the relevant specutils doc section](https://specutils.readthedocs.io/en/latest/fitting.html#continuum-fitting), but here we outline the two basic options as it stands: an "often good-enough" function, and a more customizable tool that leans on the [`astropy.modeling`](http://docs.astropy.org/en/stable/modeling/index.html) models to provide its flexibility.

### The "often good-enough" way

The `fit_generic_continuum` function provides a function that is often sufficient for reasonably well-behaved continuua, particular for "quick-look" or similar applications where high precision is not that critical.  The function yields a continuum model, which can be evaluated at any spectral axis value:

In [ ]:
from specutils.fitting import fit_generic_continuum

In [ ]:
generic_continuum = fit_generic_continuum(jwst_spec)

generic_continuum_evaluated = generic_continuum(jwst_spec.spectral_axis)

plt.step(jwst_spec.spectral_axis, jwst_spec.flux)
plt.plot(jwst_spec.spectral_axis, generic_continuum_evaluated)
plt.ylim(0, 1.5);

(Note that in some versions of astropy/specutils you may see a warning that the "Model is linear in parameters" upon executing the above cell. This is not a problem unless performance is a serious concern, in which case more customization is required.)

With this model in hand, continuum-subtracted or continuum-normalized spectra can be produced using basic spectral manipulations:

In [ ]:
jwst_gencont_sub = jwst_spec - generic_continuum(jwst_spec.spectral_axis)
jwst_gencont_norm = jwst_spec / generic_continuum(jwst_spec.spectral_axis)

ax1, ax2 = plt.subplots(2, 1)[1]

ax1.step(jwst_gencont_sub.wavelength, jwst_gencont_sub.flux)
ax1.set_ylim(-50, 50)
ax1.axhline(0, color='k', ls=':')  # continuum should be at flux=0

ax2.step(jwst_gencont_norm.wavelength, jwst_gencont_norm.flux)
ax2.set_ylim(0, 2)
ax2.axhline(1, color='k', ls='--');  # continuum should be at flux=1

### The customizable way

The `fit_continuum` function operates similarly to `fit_generic_continuum`, but is meant for you to provide your favorite continuum model rather than being tailored to a specific continuum model. To see the list of models, see the [astropy.modeling documentation](http://docs.astropy.org/en/stable/modeling/index.html).

In [ ]:
from specutils.fitting import fit_continuum
from astropy.modeling import models

For example, suppose you want to use a 3rd-degree Chebyshev polynomial as your continuum model. You can use `fit_continuum` to get an object that behaves the same as for `fit_generic_continuum`:

In [ ]:
chebdeg3_continuum = fit_continuum(jwst_spec, models.Chebyshev1D(3))

generic_continuum_evaluated = generic_continuum(jwst_spec.spectral_axis)

plt.step(jwst_spec.spectral_axis, jwst_spec.flux)
plt.plot(jwst_spec.spectral_axis, chebdeg3_continuum(jwst_spec.spectral_axis))
plt.ylim(0.4, 1.1);

This then provides total flexibility.  For example, you can also try other polynomials like higher-degree Hermite polynomials:

In [ ]:
hermdeg7_continuum = fit_continuum(jwst_spec, models.Hermite1D(degree=7))
hermdeg17_continuum = fit_continuum(jwst_spec, models.Hermite1D(degree=17))

plt.step(jwst_spec.spectral_axis, jwst_spec.flux)
plt.plot(jwst_spec.spectral_axis, chebdeg3_continuum(jwst_spec.spectral_axis))
plt.plot(jwst_spec.spectral_axis, hermdeg7_continuum(jwst_spec.spectral_axis))
plt.plot(jwst_spec.spectral_axis, hermdeg17_continuum(jwst_spec.spectral_axis))
plt.ylim(0.4, 1.1);

This immediately demonstrates the tradeoffs in polynomial fitting: while the high-degree polynomials capture the wiggles of the spectrum better than the low, they also *over*-fit near the strong emission lines.

### Exercises

Try combining the `SpectralRegion` and continuum-fitting functionality to only fit the parts of the spectrum that *are* continuum (i.e. not including emission lines).  Can you do better?

Using the spectrum from the previous exercise, first subtract a continuum, then re-do your measurement.  Is it better?

## Line-Fitting

In addition to the more empirical measurements described above, `specutils` provides tools for doing spectral line fitting. The approach is akin to that for continuum modeling: models from [astropy.modeling](http://docs.astropy.org/en/stable/modeling/index.html) are fit to the spectrum, and either those models can be used directly, or their parameters.

The fitting machinery must first be given guesses for line locations. This process can be automated using functions designed to identify lines (more detail on the options is [in the docs](https://specutils.readthedocs.io/en/latest/fitting.html#line-finding)).  For data sets where these algorithms are not ideal, you may substitute your own (i.e., skip this step and start with line location guesses). 

Here we identify the three lines near the Halpha region in our spectrum, finding the lines above about a $\sim 5 \sigma$ flux threshold.  They are then output as an astropy Table:

In [ ]:
jwst_ha_contsub = jwst_spec_with_unc_ha - jwst_continuum

halpha_lines = fitting.find_lines_threshold(jwst_ha_contsub, 5)
plt.step(jwst_spec_with_unc_ha.spectral_axis, jwst_spec_with_unc_ha.flux, where='mid')
for line in halpha_lines:
    plt.axvline(line['line_center'], color='k', ls=':')

halpha_lines

Now for each of these lines, we need to fit a model. Sometimes it is sufficient to simply create a model where the center is at the line and excise the appropriate area of the line to do a  line estimate.  This is not *too* sensitive to the size of the region, at least for well-separated lines like these.  The result is a list of models that carry with them them the details of the fit:

In [ ]:
halpha_line_models = []
for line in halpha_lines:
    line_region = SpectralRegion(line['line_center']-50*u.angstrom,
                                 line['line_center']+50*u.angstrom)
    line_spectrum = manipulation.extract_region(jwst_ha_contsub, line_region)
    line_spectrum
    
    # here's the workaround from above again
    line_spectrum = Spectrum1D(flux=line_spectrum.flux, 
                               spectral_axis=line_spectrum.spectral_axis, 
                               uncertainty=line_spectrum.uncertainty)
    line_estimate = models.Gaussian1D(mean=line['line_center'],stddev=.001*u.um)
    line_model = fitting.fit_lines(line_spectrum, line_estimate)
    
    halpha_line_models.append(line_model)
    
plt.step(jwst_ha_contsub.spectral_axis, jwst_ha_contsub.flux, where='mid')
for line_model in halpha_line_models:
    evaluated_model = line_model(jwst_ha_contsub.spectral_axis)
    plt.plot(jwst_ha_contsub.spectral_axis, evaluated_model)  
    
halpha_line_models

For more complicated models or fits it may be better to use the `estimate_line_parameters` function instead of manually creating e.g. a `Gaussian1D` model and setting the center.  An example of this pattern is given below.

Note that we provided a default `Gaussian1D` model to the `estimate_line_parameters` function above.  This function makes reasonable guesses for `Gaussian1D`, `Voigt1D`, and `Lorentz1D`, the most common line profiles used for spectral lines, but may or may not work for other models.  See [the relevant docs section](https://specutils.readthedocs.io/en/latest/fitting.html#parameter-estimation) for more details.

In this example we also show an example of a *joint* fit of both lines at the same time.  While the difference may seems subtle, in cases of blended lines this typically provides much better fits:

In [ ]:
halpha_line_estimates = []
for line in halpha_lines:
    line_region = SpectralRegion(line['line_center']-15*u.angstrom,
                                 line['line_center']+15*u.angstrom)
    line_spectrum = manipulation.extract_region(jwst_ha_contsub, line_region)
    line_estimate = fitting.estimate_line_parameters(line_spectrum, models.Gaussian1D())
    
    halpha_line_estimates.append(line_estimate)

# this could be done more flexibly with a for loop but we are explicit here for simplicity
combined_model_estimate = halpha_line_estimates[0] + halpha_line_estimates[1]

plt.step(jwst_ha_contsub.spectral_axis, jwst_ha_contsub.flux, where='mid')
plt.plot(jwst_ha_contsub.spectral_axis, 
         combined_model_estimate(jwst_ha_contsub.spectral_axis))  

combined_model_estimate

While these estimates are already pretty decent, to complete the story (especially for blended lines) the last step is to then fit the combined model:

In [ ]:
combined_model = fitting.fit_lines(jwst_ha_contsub, combined_model_estimate)

plt.step(jwst_ha_contsub.spectral_axis, jwst_ha_contsub.flux, where='mid')
plt.plot(jwst_ha_contsub.spectral_axis, 
         combined_model(jwst_ha_contsub.spectral_axis))  
    
combined_model

### Exercise

Fit a spectral feature from your own spectrum using the fitting methods outlined above. Try the different line profile types (Gaussian, Lorentzian, or Voigt).  If you are using the blackbody spectrum (where you know the "true" answer for the spectral line), compare your answer to the true answer.

## Additional Manipulations

More complex manipulation tools exist, outlined in the [relevant doc section](https://specutils.readthedocs.io/en/latest/manipulation.html). As a final example of this, we smooth our example spectrum using Gaussian smoothing:

In [ ]:
from specutils.manipulation import gaussian_smooth

smoothed_spec = gaussian_smooth(jwst_spec, 0.75)  # 0.75 pixel gaussian kernel
plt.step(smoothed_spec.wavelength, smoothed_spec.flux)

### Exercise

Try smoothing your spectrum loaded in the first exercise, or the JWST spectrum.  Compare all available kernel types (see the `convolution_smooth()` function) and decide which seems most appropriate for your spectrum.

## Additional Exercises

Create a `Spectrum1D` object for an ideal 5800 K blackbody and plot it. Try the same, but with (random) noise added and stored as the uncertainty.

Hint: while you can do this manually if you know the Planck function, there is an Astropy function to help you with this - you can find it via appropriate searches in the [astropy docs](http://docs.astropy.org).

Take the blackbody spectrum you generated above, and create a "spectral feature" by adding a Gaussian absorption or emission line to it using the arithmetic operators demonstrated above.  (Hint: [astropy.modeling](http://docs.astropy.org/en/stable/modeling/) contains [an implementation of the Gaussian line profile](http://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian1D.html#astropy.modeling.functional_models.Gaussian1D) which you may find useful.)

Now, try to make line measurements as described above on your model spectral feature.  Try varying the amplitude and see how well you recover the line's properties as it sinks down into the noise.